# House Price Predictor

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam

In [ ]:
# Load the data
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

In [ ]:
# Data preprocessing
def preprocess_data(df):
    # Handle missing values
    numeric_columns = df.select_dtypes(include=['int64', 'float64']).columns
    df[numeric_columns] = df[numeric_columns].fillna(df[numeric_columns].mean())
    
    # Convert categorical variables to dummy variables
    categorical_columns = df.select_dtypes(include=['object']).columns
    df = pd.get_dummies(df, columns=categorical_columns)
    return df

In [ ]:
# Preprocess both train and test data
X_train = preprocess_data(train_data.drop('SalePrice', axis=1))
y_train = train_data['SalePrice']
X_test = preprocess_data(test_data)

In [ ]:
# Ensure train and test have same columns
common_columns = list(set(X_train.columns) & set(X_test.columns))
X_train = X_train[common_columns]
X_test = X_test[common_columns]

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# Build the neural network model
def create_model(input_dim):
    model = Sequential([
        Dense(128, activation='relu', input_dim=input_dim),
        Dropout(0.3),
        Dense(64, activation='relu'),
        Dropout(0.2),
        Dense(32, activation='relu'),
        Dense(1)
    ])
    return model

In [ ]:
 # Create and compile the model
model = create_model(X_train_scaled.shape[1])
model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')

# Train the model
history = model.fit(
    X_train_scaled, 
    y_train,
    validation_split=0.2,
    epochs=100,
    batch_size=32,
    verbose=1
)

In [ ]:
# Make predictions on test data
predictions = model.predict(X_test_scaled)

In [ ]:
# Create submission file
submission = pd.DataFrame({
    'ID': test_data['Id'],
    'SALEPRICE': predictions.flatten()
})

# Save predictions to CSV
submission.to_csv('predictions.csv', index=False)

print("Predictions have been saved to predictions.csv")

